In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/NLP_TP/proj/data_books.csv', encoding='utf-8')

In [ ]:
data

,ISBN,Book-Title,Book-Author,Publisher,Image-URL-L,description
0,0875296459,Still Married After All These Years,Shoebox Greetings,Independent Publishers+group,http://images.amazon.com/images/P/0875296459.0...,The book is a memoir of the author's life as a...
1,0609800116,Your Heart's Desire: Instructions for Creating...,Sonia Choquette,Three Rivers Press (CA),http://images.amazon.com/images/P/0609800116.0...,Your Heart's Desire is a book about creating t...
2,0679806636,Richard Scarry's Cars and Trucks from A to Z (...,RICHARD SCARRY,Random House Books for Young Readers,http://images.amazon.com/images/P/0679806636.0...,This book is a great way to teach your child t...
3,0670049182,The House of Blue Mangoes,David Davidar,Not Avail,http://images.amazon.com/images/P/0670049182.0...,The House of Blue Mangoes is a novel by A.L. K...
4,0394832388,Early Words,Richard Scarry,Random House Children's Books,http://images.amazon.com/images/P/0394832388.0...,This book is all about teaching your baby thei...
...,...,...,...,...,...,...
58467,9781477594247,Unbelievable,Sherry Gammon (Goodreads Author),Wordpaintings Unlimited,https://i.gr-assets.com/images/S/compressed.ph...,Lilah Lopez Dreser's in town to take care of u...
58468,2940012616562,Fractured,Cheri Schmidt (Goodreads Author),Cheri Schmidt,https://i.gr-assets.com/images/S/compressed.ph...,The Fateful Trilogy continues with Fractured. ...
58469,9781461017097,Marked,Kim Richardson (Goodreads Author),CreateSpace,https://i.gr-assets.com/images/S/compressed.ph...,--READERS FAVORITE AWARDS WINNER 2011--Sixteen...
58470,9781450755634,Wayward Son,"Tom Pollack (Goodreads Author), John Loftus (G...",Cascada Productions,https://i.gr-assets.com/images/S/compressed.ph...,A POWERFUL TREMOR UNEARTHS AN ANCIENT SECRETBu...


In [ ]:
def clean_text(text):
  if isinstance(text, str):
    # Supprimer la ponctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Supprimer les chiffres
    text = re.sub(r'\d+', '', text)
    # Normaliser la casse
    text = text.lower()
    # Tokeniser le texte en mots
    words = nltk.word_tokenize(text)
    # Supprimer les stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    # Lemmatiser les mots
    wnl = WordNetLemmatizer()
    words = [wnl.lemmatize(w) for w in words]
    # Retourner les mots sous forme de chaîne de caractères
    return ' '.join(words)

In [ ]:
# Supprimer les lignes avec des NaN
data.dropna(inplace=True) 

In [ ]:
title =data["Book-Title"].apply(clean_text)
descr =data["description"].apply(clean_text)
data["text"]=title+''+descr


In [ ]:
#data.to_csv("/content/drive/MyDrive/NLP_TP/proj/clean_data.csv")

In [ ]:
#data=pd.read_csv("/content/drive/MyDrive/NLP_TP/proj/clean_data.csv")

In [ ]:
data

,ISBN,Book-Title,Book-Author,Publisher,Image-URL-L,description,text
0,0875296459,Still Married After All These Years,Shoebox Greetings,Independent Publishers+group,http://images.amazon.com/images/P/0875296459.0...,The book is a memoir of the author's life as a...,still married yearbook memoir author life wife...
1,0609800116,Your Heart's Desire: Instructions for Creating...,Sonia Choquette,Three Rivers Press (CA),http://images.amazon.com/images/P/0609800116.0...,Your Heart's Desire is a book about creating t...,heart desire instruction creating life really ...
2,0679806636,Richard Scarry's Cars and Trucks from A to Z (...,RICHARD SCARRY,Random House Books for Young Readers,http://images.amazon.com/images/P/0679806636.0...,This book is a great way to teach your child t...,richard scarrys car truck z chunky shape bookb...
3,0670049182,The House of Blue Mangoes,David Davidar,Not Avail,http://images.amazon.com/images/P/0670049182.0...,The House of Blue Mangoes is a novel by A.L. K...,house blue mangohouse blue mango novel al kenn...
4,0394832388,Early Words,Richard Scarry,Random House Children's Books,http://images.amazon.com/images/P/0394832388.0...,This book is all about teaching your baby thei...,early wordbook teaching baby first word packed...
...,...,...,...,...,...,...,...
58467,9781477594247,Unbelievable,Sherry Gammon (Goodreads Author),Wordpaintings Unlimited,https://i.gr-assets.com/images/S/compressed.ph...,Lilah Lopez Dreser's in town to take care of u...,unbelievablelilah lopez dresers town take care...
58468,2940012616562,Fractured,Cheri Schmidt (Goodreads Author),Cheri Schmidt,https://i.gr-assets.com/images/S/compressed.ph...,The Fateful Trilogy continues with Fractured. ...,fracturedfateful trilogy continues fractured c...
58469,9781461017097,Marked,Kim Richardson (Goodreads Author),CreateSpace,https://i.gr-assets.com/images/S/compressed.ph...,--READERS FAVORITE AWARDS WINNER 2011--Sixteen...,markedreader favorite award winner sixteen yea...
58470,9781450755634,Wayward Son,"Tom Pollack (Goodreads Author), John Loftus (G...",Cascada Productions,https://i.gr-assets.com/images/S/compressed.ph...,A POWERFUL TREMOR UNEARTHS AN ANCIENT SECRETBu...,wayward sonpowerful tremor unearths ancient se...


In [ ]:
data.dropna(inplace=True) 

In [ ]:
vectorizer = TfidfVectorizer()
book_vectors = vectorizer.fit_transform(data["text"])

In [ ]:
# Création de la colonne "Vecteur de description"
data['Vecteur de description'] = [vec for vec in book_vectors]


In [ ]:
data

,ISBN,Book-Title,Book-Author,Publisher,Image-URL-L,description,text,Vecteur de description
0,0875296459,Still Married After All These Years,Shoebox Greetings,Independent Publishers+group,http://images.amazon.com/images/P/0875296459.0...,The book is a memoir of the author's life as a...,still married yearbook memoir author life wife...,"(0, 46013)\t0.18258230617989418\n (0, 22944..."
1,0609800116,Your Heart's Desire: Instructions for Creating...,Sonia Choquette,Three Rivers Press (CA),http://images.amazon.com/images/P/0609800116.0...,Your Heart's Desire is a book about creating t...,heart desire instruction creating life really ...,"(0, 1728)\t0.19540010865733815\n (0, 211882..."
2,0679806636,Richard Scarry's Cars and Trucks from A to Z (...,RICHARD SCARRY,Random House Books for Young Readers,http://images.amazon.com/images/P/0679806636.0...,This book is a great way to teach your child t...,richard scarrys car truck z chunky shape bookb...,"(0, 323)\t0.20031609303463935\n (0, 159043)..."
3,0670049182,The House of Blue Mangoes,David Davidar,Not Avail,http://images.amazon.com/images/P/0670049182.0...,The House of Blue Mangoes is a novel by A.L. K...,house blue mangohouse blue mango novel al kenn...,"(0, 14992)\t0.12862098750752152\n (0, 95123..."
4,0394832388,Early Words,Richard Scarry,Random House Children's Books,http://images.amazon.com/images/P/0394832388.0...,This book is all about teaching your baby thei...,early wordbook teaching baby first word packed...,"(0, 159021)\t0.19840467230136924\n (0, 1233..."
...,...,...,...,...,...,...,...,...
58467,9781477594247,Unbelievable,Sherry Gammon (Goodreads Author),Wordpaintings Unlimited,https://i.gr-assets.com/images/S/compressed.ph...,Lilah Lopez Dreser's in town to take care of u...,unbelievablelilah lopez dresers town take care...,"(0, 77882)\t0.2525802893374533\n (0, 294569..."
58468,2940012616562,Fractured,Cheri Schmidt (Goodreads Author),Cheri Schmidt,https://i.gr-assets.com/images/S/compressed.ph...,The Fateful Trilogy continues with Fractured. ...,fracturedfateful trilogy continues fractured c...,"(0, 73079)\t0.21762444411958878\n (0, 31338..."
58469,9781461017097,Marked,Kim Richardson (Goodreads Author),CreateSpace,https://i.gr-assets.com/images/S/compressed.ph...,--READERS FAVORITE AWARDS WINNER 2011--Sixteen...,markedreader favorite award winner sixteen yea...,"(0, 173645)\t0.22810065077298816\n (0, 4118..."
58470,9781450755634,Wayward Son,"Tom Pollack (Goodreads Author), John Loftus (G...",Cascada Productions,https://i.gr-assets.com/images/S/compressed.ph...,A POWERFUL TREMOR UNEARTHS AN ANCIENT SECRETBu...,wayward sonpowerful tremor unearths ancient se...,"(0, 249813)\t0.1680524490250909\n (0, 26261..."


In [ ]:
import pickle
# Enregistrer le vecteur dans un fichier .pkl
with open('vectorizer.pkl','wb') as f:
    pickle.dump(vectorizer, f)

In [ ]:
nbrs = NearestNeighbors(n_neighbors=5, algorithm='auto').fit(book_vectors)

In [ ]:
#with open('model.pkl','wb') as f:    
 #   pickle.dump(nbrs, f)

In [ ]:
titles=data["Book-Title"]
authors=data["Book-Author"]
description=data["description"]

In [ ]:
! pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 35.0 MB/s eta 0:00:00


In [ ]:
from spellchecker import SpellChecker

# Créez un objet SpellChecker
spell = SpellChecker()

def correct_query(query):
    # Séparez la requête en mots individuels
    words = query.split()
    # Créez une liste pour stocker les mots corrigés
    corrected_words = []
    # Parcourez chaque mot dans la requête
    for word in words:
        # Si le mot n'est pas correctement orthographié, corrigez-le
        if not spell.correction(word) == word:
            corrected_words.append(spell.correction(word))
        else:
            corrected_words.append(word)
    # Rejoignez les mots corrigés pour former une requête corrigée
    corrected_query = ' '.join(corrected_words)
    # Renvoyez la requête corrigée
    return corrected_query

In [ ]:
def search_books(query):
  # Corrigez la requête    
  query = correct_query(query)
  query = clean_text(query)

# Vectoriser la requête
  query_vector = vectorizer.transform([query])
# Trouver les indices des livres les plus similaires à la requête
  distances, indices = nbrs.kneighbors(query_vector)
# Renvoyer les titres et auteurs des livres correspondants
  results = []
  for i in range(len(distances[0])):
    results.append((titles[indices[0][i]], authors[indices[0][i]],description[indices[0][i]]))
  return results

In [ ]:
search_books('book about crminal defense lawyer solving myseries involving murder and theft')


[('Perry Mason: Seven Complete Mysteries',
  'Erle S. Gardner',
  'Perry Mason: Seven Complete Mysteries contains seven full-length mysteries featuring the world-famous criminal defense lawyer. In these stories, Mason takes on cases involving murder, theft, and other crimes, using his keen intellect and vast knowledge of the law to get his clients off the hook.'),
 ('PLEADING GUILTY',
  'Scott Turow',
  "When Mallory Kane, a top criminal defense lawyer, is asked to defend her ex-husband, she knows it's a setup. But she takes the case, and quickly realizes that she is in way over her head. With her life and career on the line, she must find a way to save herself and her client."),
 ('Indemnity Only',
  'Paretsky',
  'This book is a legal thriller about a female insurance investigator who becomes involved in a case involving a powerful lawyer.'),
 ('Guilty as Sin',
  'TAMI HOAG',
  'In "Guilty as Sin," attorney and criminal defense lawyer Edward Klein paints a portrait of Hillary Clinton